In [ ]:
# Standard
import os
import sys
import datetime
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import tensorflow.keras.backend as K
from tensorflow import argmax
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

# Utils
import h5py

In [ ]:
f = h5py.File("../input/camus-dataset/image_dataset.hdf5", "r")

In [ ]:
frames2ch = f["train 2ch frames"][:,:,:,:]
masks2ch = f["train 2ch masks"][:,:,:,:]

In [ ]:
train_frames, test_frames, train_masks, test_masks = train_test_split(frames2ch, masks2ch)

# Loss Funct

In [ ]:
def generalized_dice_loss(y_true, y_pred, smooth=1e-7, num_classes=4):

    y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=4)[...,1:])
    y_pred_f = K.flatten(y_pred[...,1:])
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    denom = K.sum(y_true_f + y_pred_f, axis=-1)
    return 1.0-K.mean((2. * intersect / (denom + smooth)))

# Precision And Recall

In [ ]:
def precision_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = np.mean(intersect/total_pixel_pred)
    return round(precision, 3)

def recall_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = np.mean(intersect/total_pixel_truth)
    return round(recall, 3)

# UNet Implementation

In [ ]:
drop = 0.25

def ResBlock(input_tensor, filters):
    """"
    CONV => BATCH => RELU => DROPOUT => CONV => BATCH
      |                                   |
      |----------- SHARED LAYER ----------|
      
    """
        
    conv_1 = Conv2D(filters = filters, kernel_size = 3, padding = 'same', kernel_initializer = 'he_normal')    
    conv_1a = conv_1(input_tensor) # Shared weights conv layer
    batch_1 = BatchNormalization()(conv_1a)
    relu_1  = Activation("relu")(batch_1)
    drop_1  = Dropout(drop)(relu_1)
    conv_1b = conv_1(drop_1) # Shared weights conv layer
    batch_1 = BatchNormalization()(conv_1b)
    return batch_1

def UNet(input_size = (256, 256, 1), num_classes=2, filters=30):
    
    # X's denote standard flow
    # XNUM denote ResBlock outputs
    
    # "First" UNet
    
    # Input branch
    inputs = Input(input_size)
    X = Conv2D(filters=filters, kernel_size=3, activation="relu", padding = 'same', kernel_initializer = 'he_normal')(inputs)

    # Down branch
    X1 = ResBlock(input_tensor=X, filters=filters) # ResBlock located in the first layer of the paper scheme
    X = Conv2D(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X1) 
    X = Activation("relu")(X) # This ReLU is not shown in the paper scheme
    
    X2 = ResBlock(input_tensor=X, filters=filters*2)
    X = Conv2D(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X2)
    X = Activation("relu")(X)
    
    X3 = ResBlock(input_tensor=X, filters=filters*4)
    X = Conv2D(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X3)
    X = Activation("relu")(X)
    
    X4 = ResBlock(input_tensor=X, filters=filters*8)
    X = Conv2D(filters=filters*16, kernel_size=3, strides=2, kernel_initializer='he_normal')(X4)
    X = Activation("relu")(X)
    
    # Bottom block 
    X = ResBlock(input_tensor=X, filters=filters*16)
    
    # Up branch
    X = Conv2DTranspose(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X4])
    # X = Activation("relu")(X) # This ReLU is commented in the paper code
    X5 = ResBlock(input_tensor=X, filters=filters*8)
    
    X = Conv2DTranspose(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X5)
    X = Add()([X, X3])
    # X = Activation("relu")(X)
    X6 = ResBlock(input_tensor=X, filters=filters*4)
    
    X = Conv2DTranspose(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X6)
    X = Add()([X, X2])
    # X = Activation("relu")(X)
    X7 = ResBlock(input_tensor=X, filters=filters*2)
        
    X = Conv2DTranspose(filters=filters, kernel_size=3, strides=2, output_padding=1, kernel_initializer='he_normal')(X7)
    X = Add()([X, X1])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters)
    
    # Top block (bottle-neck)
    X8 = ResBlock(input_tensor=X, filters=filters)
    X = ResBlock(input_tensor=X, filters=filters)
    X = Add()([X, X8])
    
    # "Second" UNet
    
    # Down branch
    X9 = ResBlock(input_tensor=X, filters=filters)
    X = Conv2D(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X) 
    X = Activation("relu")(X)
    X = Add()([X7, X])    
    
    X10 = ResBlock(input_tensor=X, filters=filters*2)
    X = Conv2D(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)    
    X = Add()([X6, X])
    
    X11 = ResBlock(input_tensor=X, filters=filters*4)
    X = Conv2D(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)
    X = Add()([X5, X])

    X12 = ResBlock(input_tensor=X, filters=filters*8)
    X = Conv2D(filters=filters*16, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)    
    X = Activation("relu")(X)
    
    # Bottom block
    X = ResBlock(input_tensor=X, filters=filters*16)
    
    # Up branch
    X = Conv2DTranspose(filters=filters*8, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X12])   
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters*8)
    
    X = Conv2DTranspose(filters=filters*4, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X11])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters*4)
    
    X = Conv2DTranspose(filters=filters*2, kernel_size=3, strides=2, kernel_initializer='he_normal')(X)
    X = Add()([X, X10])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters*2)
    
    X = Conv2DTranspose(filters=filters, kernel_size=3, strides=2, kernel_initializer='he_normal', output_padding=1)(X)
    X = Add()([X, X9])
    # X = Activation("relu")(X)
    X = ResBlock(input_tensor=X, filters=filters)
    
    # Final block
    X = Conv2D(filters=num_classes, kernel_size=1, kernel_initializer='he_normal')(X)
    # X = Activation("relu")(X)
    X = Activation("softmax")(X)
    #X = Conv2D(1, 1)(X)
    
    model = Model(inputs, X)
    
    return model

In [ ]:
model_Unet = UNet(input_size=(384, 384, 1), num_classes=4, filters=20)

In [ ]:
earlystop = EarlyStopping(monitor='val_multiclass_dice', min_delta=0, patience=5,
                          verbose=1, mode="max", restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor='val_multiclass_dice', factor=0.2, patience=2,
                              verbose=1, mode="max", min_lr=1e-5)

In [ ]:
model_Unet.summary()

In [ ]:
model_Unet.compile(optimizer=Adam(lr=1e-3), loss="sparse_categorical_crossentropy", metrics=[multiclass_dice, "accuracy"])

In [ ]:
history = model_Unet.fit(x=train_frames,
                    y=train_masks,
                    validation_data=[test_frames, test_masks],
                    batch_size=5,
                    epochs=50,
                    callbacks=[earlystop, reduce_lr])

In [ ]:
precision = precision_score_(train_masks, test_masks)
recall = recall_score_(train_masks, test_masks)

print('The Precision is: ', precision)
print('The Recall is: ', recall)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 10
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 24))
    for i in range(10):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.show()


predicted_masks = []
for i in range(10):
    prediction = model_Unet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
    
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")
plot_graphs(history, "multiclass_dice")

# For 4ch

In [ ]:
frames4ch = f["train 4ch frames"][:,:,:,:]
masks4ch = f["train 4ch masks"][:,:,:,:]

In [ ]:
train_frames, test_frames, train_masks, test_masks = train_test_split(frames4ch, masks4ch)

In [ ]:
train_frames.shape

In [ ]:
model_Unet = UNet(input_size=(384, 384, 1), num_classes=4, filters=20)

In [ ]:
earlystop = EarlyStopping(monitor='val_multiclass_dice', min_delta=0, patience=5,
                          verbose=1, mode="max", restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor='val_multiclass_dice', factor=0.2, patience=2,
                              verbose=1, mode="max", min_lr=1e-5)

In [ ]:
model_Unet.compile(optimizer=Adam(lr=1e-3), loss="sparse_categorical_crossentropy", metrics=[multiclass_dice, "accuracy"])

history = model_Unet.fit(x=train_frames,
                    y=train_masks,
                    validation_data=[test_frames, test_masks],
                    batch_size=5,
                    epochs=50,
                    callbacks=[earlystop, reduce_lr])

In [ ]:
precision = precision_score_(train_masks, test_masks)
recall = recall_score_(train_masks, test_masks)

print('The Precision is: ', precision)
print('The Recall is: ', recall)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 10
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 24))
    for i in range(10):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.show()


predicted_masks = []
for i in range(10):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
    
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")
plot_graphs(history, "multiclass_dice")

# Calculating Subject-based Validation - Different approach from the original study

In [ ]:
import json, codecs

def saveHist(path, history):
    with codecs.open(path, 'w', encoding='utf-8') as f:
        json.dump(history, f, separators=(',', ':'), sort_keys=True, indent=4) 

def loadHist(path):
    n = {} # set history to empty
    if os.path.exists(path): # reload history if it exists
        with codecs.open(path, 'r', encoding='utf-8') as f:
            n = json.loads(f.read())
    return n

def appendHist(h1, h2):
    if h1 == {}:
        return h2
    else:
        dest = {}
        for key, value in h1.items():
            dest[key] = value + h2[key]
        return dest

In [ ]:
f = h5py.File("../input/camus-subject-based/subject_based_dataset.hdf5", "r")

In [ ]:
frames2ched = f["train 2ch ED frames"][:,:,:,:]
masks2ched = f["train 2ch ED masks"][:,:,:,:]
train_frames, test_frames, train_masks, test_masks = train_test_split(frames2ched, masks2ched, random_state=42)

In [ ]:
model_laddernet = LadderNet(input_size=(384, 384, 1), num_classes=4, filters=20)

earlystop = EarlyStopping(monitor='val_multiclass_dice', min_delta=0, patience=5,
                          verbose=1, mode="max", restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor='val_multiclass_dice', factor=0.2, patience=2,
                              verbose=1, mode="max", min_lr=1e-5)

model_laddernet.compile(optimizer=Adam(lr=1e-3), loss="sparse_categorical_crossentropy", metrics=[multiclass_dice, "accuracy"])

history = model_laddernet.fit(x=train_frames,
                    y=train_masks,
                    validation_data=[test_frames, test_masks],
                    batch_size=8,
                    epochs=30,
                    callbacks=[earlystop, reduce_lr])

In [ ]:
precision = precision_score_(train_masks, test_masks)
recall = recall_score_(train_masks, test_masks)

print('The Precision is: ', precision)
print('The Recall is: ', recall)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 6
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 18))
    for i in range(6):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.savefig("2chED.png")

    plt.show()


predicted_masks = []
for i in range(6):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
    
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
cols = 3
rows = 1
figure, (ax1,ax2,ax3) = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 4))

ax1.plot(history.history["accuracy"])
ax1.plot(history.history['val_accuracy'])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.legend(["training_accuracy", 'val_accuracy'])

ax2.plot(history.history["loss"])
ax2.plot(history.history['val_loss'])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.legend(["training_loss", 'val_loss'])

ax3.plot(history.history["multiclass_dice"])
ax3.plot(history.history['val_multiclass_dice'])
ax3.set_xlabel("Epochs")
ax3.set_ylabel("Dice Loss")
ax3.legend(["training_dice_loss",'val_dice_loss'])

ax1.set_title("Fig. Accuracy")
ax2.set_title("Fig. Loss")
ax3.set_title("Fig. Multiclass Dice")
#ax1.set_axis_off()
#ax2.set_axis_off()
#ax3.set_axis_off()
plt.tight_layout()
plt.savefig("history1.png")
plt.show()

----

In [ ]:
frames2ches = f["train 2ch ES frames"][:,:,:,:]
masks2ches = f["train 2ch ES masks"][:,:,:,:]
train_frames, test_frames, train_masks, test_masks = train_test_split(frames2ches, masks2ches, random_state=42)

In [ ]:
history2 = model_laddernet.fit(x=train_frames,
                    y=train_masks,
                    validation_data=[test_frames, test_masks],
                    batch_size=8,
                    epochs=30,
                    callbacks=[earlystop, reduce_lr])

In [ ]:
precision = precision_score_(train_masks, test_masks)
recall = recall_score_(train_masks, test_masks)

print('The Precision is: ', precision)
print('The Recall is: ', recall)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 6
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 18))
    for i in range(6):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.savefig("2chES.png")

    plt.show()


predicted_masks = []
for i in range(6):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
    
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
cols = 3
rows = 1
figure, (ax1,ax2,ax3) = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 4))

ax1.plot(history2.history["accuracy"])
ax1.plot(history2.history['val_accuracy'])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.legend(["training_accuracy", 'val_accuracy'])

ax2.plot(history2.history["loss"])
ax2.plot(history2.history['val_loss'])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.legend(["training_loss", 'val_loss'])

ax3.plot(history2.history["multiclass_dice"])
ax3.plot(history2.history['val_multiclass_dice'])
ax3.set_xlabel("Epochs")
ax3.set_ylabel("Dice Loss")
ax3.legend(["training_dice_loss",'val_dice_loss'])

ax1.set_title("Fig. Accuracy")
ax2.set_title("Fig. Loss")
ax3.set_title("Fig. Multiclass Dice")
#ax1.set_axis_off()
#ax2.set_axis_off()
#ax3.set_axis_off()
plt.tight_layout()
plt.savefig("history2.png")
plt.show()

----

In [ ]:
frames4ched = f["train 4ch ED frames"][:,:,:,:]
masks4ched = f["train 4ch ED masks"][:,:,:,:]
train_frames, test_frames, train_masks, test_masks = train_test_split(frames4ched, masks4ched, random_state=42)

In [ ]:
history3 = model_laddernet.fit(x=train_frames,
                    y=train_masks,
                    validation_data=[test_frames, test_masks],
                    batch_size=8,
                    epochs=30,
                    callbacks=[earlystop, reduce_lr])

In [ ]:
precision = precision_score_(train_masks, test_masks)
recall = recall_score_(train_masks, test_masks)

print('The Precision is: ', precision)
print('The Recall is: ', recall)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 6
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 18))
    for i in range(6):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.savefig("4chED.png")
    plt.show()


predicted_masks = []
for i in range(6):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
    
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
cols = 3
rows = 1
figure, (ax1,ax2,ax3) = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 4))

ax1.plot(history3.history["accuracy"])
ax1.plot(history3.history['val_accuracy'])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.legend(["training_accuracy", 'val_accuracy'])

ax2.plot(history3.history["loss"])
ax2.plot(history3.history['val_loss'])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.legend(["training_loss", 'val_loss'])

ax3.plot(history3.history["multiclass_dice"])
ax3.plot(history3.history['val_multiclass_dice'])
ax3.set_xlabel("Epochs")
ax3.set_ylabel("Dice Loss")
ax3.legend(["training_dice_loss",'val_dice_loss'])

ax1.set_title("Fig. Accuracy")
ax2.set_title("Fig. Loss")
ax3.set_title("Fig. Multiclass Dice")
#ax1.set_axis_off()
#ax2.set_axis_off()
#ax3.set_axis_off()
plt.tight_layout()
plt.savefig("history3.png")
plt.show()

----

In [ ]:
frames4ches = f["train 4ch ES frames"][:,:,:,:]
masks4ches = f["train 4ch ES masks"][:,:,:,:]
train_frames, test_frames, train_masks, test_masks = train_test_split(frames4ches, masks4ches, random_state=42)

In [ ]:
history4 = model_laddernet.fit(x=train_frames,
                    y=train_masks,
                    validation_data=[test_frames, test_masks],
                    batch_size=8,
                    epochs=30,
                    callbacks=[earlystop, reduce_lr])

In [ ]:
precision = precision_score_(train_masks, test_masks)
recall = recall_score_(train_masks, test_masks)

print('The Precision is: ', precision)
print('The Recall is: ', recall)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 6
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 18))
    for i in range(6):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.savefig("4chES.png")
    plt.show()


predicted_masks = []
for i in range(6):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
    
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
cols = 3
rows = 1
figure, (ax1,ax2,ax3) = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 4))

ax1.plot(history4.history["accuracy"])
ax1.plot(history4.history['val_accuracy'])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.legend(["training_accuracy", 'val_accuracy'])

ax2.plot(history4.history["loss"])
ax2.plot(history4.history['val_loss'])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.legend(["training_loss", 'val_loss'])

ax3.plot(history4.history["multiclass_dice"])
ax3.plot(history4.history['val_multiclass_dice'])
ax3.set_xlabel("Epochs")
ax3.set_ylabel("Dice Loss")
ax3.legend(["training_dice_loss",'val_dice_loss'])

ax1.set_title("Fig. Accuracy")
ax2.set_title("Fig. Loss")
ax3.set_title("Fig. Multiclass Dice")
#ax1.set_axis_off()
#ax2.set_axis_off()
#ax3.set_axis_off()
plt.tight_layout()
plt.savefig("history4.png")
plt.show()

In [ ]:
print(history.history.keys())

In [ ]:
all_accuracy=np.concatenate((history.history['accuracy'],history2.history['accuracy'], history3.history['accuracy'], history4.history['accuracy']),axis=0)
all_val_accuracy=np.concatenate((history.history['val_accuracy'], history2.history['val_accuracy'], history3.history['val_accuracy'], history4.history['val_accuracy']),axis=0)
all_loss=np.concatenate((history.history['loss'],history2.history['loss'], history3.history['loss'],history4.history['loss']),axis=0)
all_val_loss=np.concatenate((history.history['val_loss'],history2.history['val_loss'], history3.history['val_loss'],history4.history['val_loss']),axis=0)
all_diceloss=np.concatenate((history.history['multiclass_dice'],history2.history['multiclass_dice'], history3.history['multiclass_dice'],history4.history['multiclass_dice']),axis=0)
all_val_diceloss=np.concatenate((history.history['val_multiclass_dice'],history2.history['val_multiclass_dice'], history3.history['val_multiclass_dice'],history4.history['val_multiclass_dice']),axis=0)

In [ ]:
cols = 3
rows = 1
figure, (ax1,ax2,ax3) = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 4))

ax1.plot(all_accuracy)
ax1.plot(all_val_accuracy)
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.legend(["training_accuracy", 'val_accuracy'])

ax2.plot(all_loss)
ax2.plot(all_val_loss)
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.legend(["training_loss", 'val_loss'])

ax3.plot(all_diceloss)
ax3.plot(all_val_diceloss)
ax3.set_xlabel("Epochs")
ax3.set_ylabel("Dice Loss")
ax3.legend(["training_dice_loss",'val_dice_loss'])

ax1.set_title("Accuracy")
ax2.set_title("Loss")
ax3.set_title("Multiclass Dice")
#ax1.set_axis_off()
#ax2.set_axis_off()
#ax3.set_axis_off()
plt.tight_layout()
plt.savefig("history_all.png")
plt.show()

In [ ]:
frames2ched = f["train 2ch ED frames"][:,:,:,:]
masks2ched = f["train 2ch ED masks"][:,:,:,:]
train_frames, test_frames, train_masks, test_masks = train_test_split(frames2ched, masks2ched, random_state=42)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 6
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 18))
    for i in range(6):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()
        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.savefig("2chED_AfterAllTrain.png")

    plt.show()


predicted_masks = []
for i in range(6):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
    
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
frames2ches = f["train 2ch ES frames"][:,:,:,:]
masks2ches = f["train 2ch ES masks"][:,:,:,:]
train_frames, test_frames, train_masks, test_masks = train_test_split(frames2ches, masks2ches, random_state=42)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 6
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 18))
    for i in range(6):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.savefig("2chES_AfterAllTrain.png")
    plt.show()


predicted_masks = []
for i in range(6):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))

display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)

In [ ]:
frames4ched = f["train 4ch ED frames"][:,:,:,:]
masks4ched = f["train 4ch ED masks"][:,:,:,:]
train_frames, test_frames, train_masks, test_masks = train_test_split(frames4ched, masks4ched, random_state=42)

In [ ]:
def display_image_grid(test_frames, test_masks, predicted_masks=None):
    cols = 3 if predicted_masks else 2
    rows = 6
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(10, 18))
    for i in range(6):
        img = test_frames[i,:,:,0]
        mask = test_masks[i,:,:,0]
        
        ax[i, 0].imshow(img, cmap='gray')
        ax[i, 1].imshow(mask, interpolation="nearest")

        ax[i, 0].set_title("Image")
        ax[i, 1].set_title("Ground truth mask")

        ax[i, 0].set_axis_off()
        ax[i, 1].set_axis_off()

        if predicted_masks:
            predicted_mask = predicted_masks[i]
            ax[i, 2].imshow(predicted_mask)
            ax[i, 2].set_title("Predicted mask")
            ax[i, 2].set_axis_off()
    plt.tight_layout()
    plt.savefig("4chED_AfterAllTrain.png")
    plt.show()


predicted_masks = []
for i in range(6):
    prediction = model_laddernet.predict(test_frames[i:i+1,:,:,:])
    prediction = prediction.reshape([384, 384, 4])
    y = tf.convert_to_tensor(prediction)
    predicted_masks.append(tf.math.argmax(prediction, axis = 2))
display_image_grid(test_frames, test_masks, predicted_masks=predicted_masks)